<a href="https://colab.research.google.com/drive/1E3519VKVwHYaDrg6CbSHBYFBXp9t8We4?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>

In [1]:
!pip install -qU google-generativeai

In [2]:
import google.generativeai as genai
import getpass
from collections import Counter

Get free-tier Google's Gemini API Key here: https://aistudio.google.com/app/apikey

In [3]:
API_KEY = getpass.getpass("Enter your Google API key: ")

Enter your Google API key: ··········


In [4]:
genai.configure(api_key=API_KEY)

In [5]:
class CoT:
    def __init__(self):
        self.model = genai.GenerativeModel("gemini-2.0-flash-exp")

    def zero_shot_cot(self, question):
        """Zero-Shot CoT: 'Let's think step by step'"""
        print(f"\n{'='*60}")
        print(f"🧠 Zero-Shot CoT")
        print(f"{'='*60}")
        print(f"Question: {question}\n")

        prompt = f"""{question}

        Let's think step by step."""

        response = self.model.generate_content(prompt).text
        print(f"Reasoning:\n{response}\n")

        return response

    def few_shot_cot(self, question, examples):
        """Few-Shot CoT: Provide examples with reasoning"""
        print(f"\n{'='*60}")
        print(f"📚 Few-Shot CoT")
        print(f"{'='*60}")
        print(f"Question: {question}\n")

        # Build prompt with examples
        examples_text = "\n\n".join([
            f"Q: {ex['question']}\nA: {ex['reasoning']}"
            for ex in examples
        ])

        prompt = f"""Here are some examples:

        {examples_text}

        Now solve this:

        Q: {question}
        A: Let's think step by step."""

        response = self.model.generate_content(prompt).text
        print(f"Reasoning:\n{response}\n")

        return response

    def self_consistency(self, question, num_paths=5):
        """Self-Consistency: Multiple reasoning paths + majority vote"""
        print(f"\n{'='*60}")
        print(f"🔄 Self-Consistency CoT ({num_paths} paths)")
        print(f"{'='*60}")
        print(f"Question: {question}\n")

        answers = []

        for i in range(num_paths):
            print(f"Path {i+1}:")

            prompt = f"""{question}

            Let's think step by step."""

            response = self.model.generate_content(prompt).text

            # Extract final answer (last line or number)
            lines = response.strip().split("\n")
            final_answer = lines[-1].strip()

            # Try to extract just the answer value
            import re
            numbers = re.findall(r'\d+\.?\d*', final_answer)
            if numbers:
                final_answer = numbers[-1]

            answers.append(final_answer)
            print(f"  Answer: {final_answer}\n")

        # Majority vote
        vote_counts = Counter(answers)
        majority_answer = vote_counts.most_common(1)[0][0]

        print(f"{'─'*60}")
        print(f"📊 Vote Results:")
        for answer, count in vote_counts.items():
            print(f"  {answer}: {count} votes")
        print(f"\n🏆 Majority Answer: {majority_answer}\n")

        return majority_answer

    def compare_methods(self, question, examples=None):
        """Compare all three CoT methods"""
        print(f"\n{'='*70}")
        print(f"🔬 COMPARING CoT METHODS")
        print(f"{'='*70}\n")

        # Method 1: Zero-Shot
        zero_shot_result = self.zero_shot_cot(question)

        # Method 2: Few-Shot (if examples provided)
        if examples:
            few_shot_result = self.few_shot_cot(question, examples)

        # Method 3: Self-Consistency
        self_consistency_result = self.self_consistency(question, num_paths=3)

        print(f"{'='*70}")
        print(f"📋 SUMMARY")
        print(f"{'='*70}")
        print(f"Zero-Shot: Check output above")
        if examples:
            print(f"Few-Shot: Check output above")
        print(f"Self-Consistency: {self_consistency_result}")
        print()

In [6]:
# Initialize
cot = CoT()

# Example 1: Math Word Problem
print("="*70)
print("EXAMPLE 1: Math Word Problem")
print("="*70)

math_question = """Roger has 5 tennis balls. He buys 2 more cans of tennis balls.
Each can has 3 tennis balls. How many tennis balls does he have now?"""

cot.zero_shot_cot(math_question)


# Example 2: Few-Shot CoT with Examples
print("\n" + "="*70)
print("EXAMPLE 2: Few-Shot CoT with Examples")
print("="*70)

examples = [
    {
        "question": "John has 3 apples. He buys 2 more. How many does he have?",
        "reasoning": "John starts with 3 apples. He buys 2 more. So 3 + 2 = 5. The answer is 5."
    },
    {
        "question": "A box has 4 red balls and 3 blue balls. How many balls total?",
        "reasoning": "Red balls: 4. Blue balls: 3. Total = 4 + 3 = 7. The answer is 7."
    }
]

new_question = "Sara has 8 cookies. She gives 3 to her friend and buys 5 more. How many cookies does Sara have now?"

cot.few_shot_cot(new_question, examples)


# Example 3: Self-Consistency
print("\n" + "="*70)
print("EXAMPLE 3: Self-Consistency CoT")
print("="*70)

tricky_question = """A farmer has 17 sheep. All but 9 die. How many sheep are left?"""

cot.self_consistency(tricky_question, num_paths=5)


# Example 4: Logical Reasoning
print("\n" + "="*70)
print("EXAMPLE 4: Logical Reasoning")
print("="*70)

logic_question = """All roses are flowers. Some flowers fade quickly.
Does this mean some roses fade quickly?"""

cot.zero_shot_cot(logic_question)


# Example 5: Commonsense Reasoning
print("\n" + "="*70)
print("EXAMPLE 5: Commonsense Reasoning")
print("="*70)

commonsense_question = """If you drop a glass vase on a concrete floor, what will happen?"""

cot.zero_shot_cot(commonsense_question)


# Example 6: Compare All Methods
print("\n" + "="*70)
print("EXAMPLE 6: Compare All Methods")
print("="*70)

comparison_question = """A restaurant has 23 tables. Each table has 4 chairs.
If 3 tables are broken and removed, how many chairs are left?"""

comparison_examples = [
    {
        "question": "A classroom has 6 rows. Each row has 5 desks. How many desks total?",
        "reasoning": "6 rows × 5 desks per row = 30 desks. The answer is 30."
    }
]

cot.compare_methods(comparison_question, comparison_examples)


# Example 7: Complex Multi-Step Problem
print("\n" + "="*70)
print("EXAMPLE 7: Complex Multi-Step Reasoning")
print("="*70)

complex_question = """A store sells notebooks for $3 each and pens for $2 each.
Sarah buys 4 notebooks and 6 pens. She pays with a $50 bill.
How much change does she receive?"""

cot.zero_shot_cot(complex_question)


print("✅ Chain-of-Thought Prompting Complete!")

EXAMPLE 1: Math Word Problem

🧠 Zero-Shot CoT
Question: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. 
Each can has 3 tennis balls. How many tennis balls does he have now?

Reasoning:
Here's how to solve the problem step-by-step:

1. **Calculate the number of tennis balls in the cans:** Roger buys 2 cans * 3 tennis balls/can = 6 tennis balls.
2. **Calculate the total number of tennis balls:** Roger has 5 tennis balls + 6 tennis balls = 11 tennis balls.

**Answer:** Roger has 11 tennis balls now.



EXAMPLE 2: Few-Shot CoT with Examples

📚 Few-Shot CoT
Question: Sara has 8 cookies. She gives 3 to her friend and buys 5 more. How many cookies does Sara have now?

Reasoning:
Sara starts with 8 cookies. She gives 3 away, so she has 8 - 3 = 5 cookies. Then she buys 5 more, so she has 5 + 5 = 10 cookies. The answer is 10.



EXAMPLE 3: Self-Consistency CoT

🔄 Self-Consistency CoT (5 paths)
Question: A farmer has 17 sheep. All but 9 die. How many sheep are left?

Path 1:
  Ans